## Énoncé
Data Scientist au sein d'une société financière, nommée "Prêt à dépenser",  qui propose des crédits à la consommation pour des personnes ayant peu ou pas du tout d'historique de prêt.
L’entreprise souhaite mettre en œuvre un outil de “scoring crédit” pour calculer la probabilité qu’un client rembourse son crédit, puis classifie la demande en crédit accordé ou refusé.
## Mission
Consultant au sein de cette entreprise, notre mission est de construire un modèle de scoring qui donnera une prédiction sur la probabilité de faillite d'un client de façon automatique.